<a href="https://colab.research.google.com/github/Computer-CGuy/BitcoinTPU/blob/main/Bitcoin_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

In [2]:
import torch_xla
import torch_xla.core.xla_model as xm

In [3]:
CORES = len(torch_xla.core.xla_model.get_xla_supported_devices())
print("Number of cores",CORES)

Number of cores 8


In [33]:
import time

# from utils import SHA256_hash

import torch
from torch.utils.data import Dataset

# device = xm.xla_device()
# device = torch.device("cuda")
F32 = 0xffffffff
base = [2863311530, 898303315, 1398735129, 0, 2147483648, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 640, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
h = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
k = torch.LongTensor(
    [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
      0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
      0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
      0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
      0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
      0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
      0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
      0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
      0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
      0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
      0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
      0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
      0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
      0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
      0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
      0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2])#.to(device)
# consts = [[7,25],[18,14],[3,29],[17,8],[19,13],[2,30],[13,19],[22,10],[6,26],[11,21],[25,7]]
consts = [[2**7,2**25],[2**18,2**14],[2**3,2**29],[2**17,2**8],[2**19,2**13],[2**2,2**30],[2**13,2**19],[2**22,2**10],[2**6,2**26],[2**11,2**21],[25,2**7]]
consts = torch.LongTensor(consts)#.to(device)
class hashing_algorithm(Dataset):
    def __init__(self,start,end,base):
        self.base = base
        self.start = start
        self.end = end
    def __len__(self):
        return self.end-self.start
    def __getitem__(self,idx):
        w = self.base
        w[3] = idx+self.start
        return torch.LongTensor(w),torch.LongTensor(h),torch.LongTensor([1]),torch.LongTensor([1])

from torch.utils.data import DataLoader
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp


In [5]:
def my_stack(batch):
    # print(len(batch))
    elem = batch[0]
    # print(len(elem))
    elem_type = type(elem)
    # print(elem_type)
    out = None
    numel = sum([x.numel() for x in batch])
    storage = elem.storage()._new_shared(numel)
    out = elem.new(storage)
    return torch.stack(batch, 1, out=out)

In [6]:
def my_collate(batch):
    return [(*(my_stack(samples) for samples in zip(*batch)))]


In [7]:
consts[3][0]

tensor(17)

In [8]:
import torch
# F32 = 0xffffffff

def _rotr(x, const,F32):
    return ((x >> const[0]) | (x << (const[1]))) & F32

def _maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

def _ch(x, y, z):
    return (x & y) ^ ((~x) & z)


def SHA256_hash(w,a,b,c,d,e,f,g,h,k,t1,t2,F32,consts):
    for i in range(16, 64):
        torch.bitwise_xor(_rotr(w[i-15], consts[0],F32), _rotr(w[i-15], consts[1],F32),out=t1)
        torch.bitwise_xor(t1,_rotr(w[i-15], consts[2],F32),out=t1)

        # s1 

        torch.bitwise_xor(_rotr(w[i-2], consts[3],F32),_rotr(w[i-2], consts[4],F32),out=t2)
        torch.bitwise_xor(t2,_rotr(w[i-15], consts[3],F32),out=t2)
        # s1[:] = _rotr(w[i-2], 17) ^ _rotr(w[i-2], 19) ^ _rotr(w[i-15], 10)
        # w[i] = (w[i-16] + t1 + w[i-7] + t2) & F32
        w[i] = (w[i-16] + t1 + w[i-7] + t2) & F32
    for i in range(64): 
        torch.bitwise_xor(_rotr(a, consts[5],F32), _rotr(a, consts[6],F32),out=t2)
        torch.bitwise_xor(t2,_rotr(a, consts[7],F32),out=t2)

        # s0[:] = _rotr(chrs[i%8], 2) ^ _rotr(chrs[i%8], 13) ^ _rotr(chrs[i%8], 22)
        torch.add(t2 ,_maj(a, b, c),out=t2)


        torch.bitwise_xor(_rotr(e, consts[8],F32), _rotr(e, consts[9],F32),out=t1)
        torch.bitwise_xor(t1,_rotr(e, consts[10],F32),out=t1)
        
        # s1[:] = _rotr(chrs[(i+4)%8], 6) ^ _rotr(chrs[(i+4)%8], 11) ^ _rotr(chrs[(i+4)%8], 25)
        torch.add(h,t1,out=t1)
        torch.add(_ch(e,f,g) + k[i],t1+w[i],out=t1)
        # t1[:] = chrs[(i+7)%8] + s1 + _ch(chrs[(i+4)%8], chrs[(i+5)%8], chrs[(i+6)%8]) + k[i] + w[i]
        
        h = g
        g = f
        f = e
        torch.add(d, t1,out=e)
        torch.bitwise_and(e, F32,out=e)
        d = c
        c = b
        b = a
        torch.add(t1, t2,out=a)
        torch.bitwise_and(a, F32,out=a)
        # a = (t1 + t2) & F32

In [9]:
# hash_dataset = hashing_algorithm(0*batch_n, (0+1)*batch_n, base)
# hash_dataloader = DataLoader(hash_dataset, batch_size=each, shuffle=False,collate_fn=my_collate)
# mp_device_loader = pl.MpDeviceLoader(hash_dataloader, device)

In [10]:
# for x in mp_device_loader:
#     break

In [11]:
# len(x)

In [12]:
F32=0xffffffff

In [13]:
import torch_xla.debug.metrics as met

print(met.metrics_report())

Metric: XrtAllocateFromTensor
  TotalSamples: 32
  Accumulator: 01s198ms229.243us
  Mean: 037ms444.664us
  StdDev: 170ms368.075us
  Rate: 0.259621 / second
  Percentiles: 25%=528.957us; 50%=789.428us; 80%=009ms102.109us; 90%=012ms705.927us; 95%=098ms099.853us; 99%=981ms541.900us
Metric: XrtCompile
  TotalSamples: 11
  Accumulator: 02s852ms934.014us
  Mean: 168ms357.638us
  StdDev: 489ms211.989us
  Rate: 5.73993 / second
  Percentiles: 25%=013ms146.591us; 50%=014ms853.015us; 80%=014ms126.024us; 90%=015ms688.174us; 95%=02s715ms380.815us; 99%=02s715ms380.815us
Metric: XrtExecute
  TotalSamples: 2016
  Accumulator: 02s480ms203.577us
  Mean: 001ms250.453us
  StdDev: 229.743us
  Rate: 49.9536 / second
  Percentiles: 25%=001ms094.586us; 50%=001ms232.074us; 80%=001ms402.427us; 90%=002ms559.056us; 95%=002ms722.613us; 99%=002ms878.421us
Metric: XrtExecutorEvict
  TotalSamples: 0
  Accumulator: nanB
  Mean: nanB
  StdDev: nanB
  Percentiles: 
Metric: XrtReadLiteral
  TotalSamples: 4032
  Accumula

In [14]:
import torch_xla.debug.metrics as met

print(met.metrics_report())

Metric: XrtAllocateFromTensor
  TotalSamples: 32
  Accumulator: 01s198ms229.243us
  Mean: 037ms444.664us
  StdDev: 170ms368.075us
  Rate: 0.259621 / second
  Percentiles: 25%=528.957us; 50%=789.428us; 80%=009ms102.109us; 90%=012ms705.927us; 95%=098ms099.853us; 99%=981ms541.900us
Metric: XrtCompile
  TotalSamples: 11
  Accumulator: 02s852ms934.014us
  Mean: 168ms357.638us
  StdDev: 489ms211.989us
  Rate: 5.73993 / second
  Percentiles: 25%=013ms146.591us; 50%=014ms853.015us; 80%=014ms126.024us; 90%=015ms688.174us; 95%=02s715ms380.815us; 99%=02s715ms380.815us
Metric: XrtExecute
  TotalSamples: 2016
  Accumulator: 02s480ms203.577us
  Mean: 001ms250.453us
  StdDev: 229.743us
  Rate: 49.9536 / second
  Percentiles: 25%=001ms094.586us; 50%=001ms232.074us; 80%=001ms402.427us; 90%=002ms559.056us; 95%=002ms722.613us; 99%=002ms878.421us
Metric: XrtExecutorEvict
  TotalSamples: 0
  Accumulator: nanB
  Mean: nanB
  StdDev: nanB
  Percentiles: 
Metric: XrtReadLiteral
  TotalSamples: 4032
  Accumula

In [18]:
def _mp_fn(index):
    global batch_n
    global each
    global start
    global k
    global consts
    print("Started",index)
    device = xm.xla_device()
    k = k.to(device)
    consts = consts.to(device)
    print("Datak")
    datak = torch.zeros([64,each],dtype=torch.int64).to(device)
    # torch.Size([8, 100])
    # torch.Size([1, 100])
    # torch.Size([1, 100])
    print("Chrs")
    chrs = torch.zeros([8,each],dtype=torch.int64).to(device)
    print("t1")
    t1 = torch.zeros([1,each],dtype=torch.int64).to(device)
    print("t2")
    t2 = torch.zeros([1,each],dtype=torch.int64).to(device)# hash_dataset = hashing_algorithm(index*each, (index+1)*each, base)
    print("done")
    # hash_dataloader = DataLoader(hash_dataset, batch_size=each, shuffle=False,collate_fn=my_collate)
    # mp_device_loader = pl.MpDeviceLoader(hash_dataloader, device)
    # # datak = torch.zeros()
    # for datak,chrs,t1,t2 in mp_device_loader:
    #     print(datak.shape)
    #     print(chrs.shape)
    #     print(t1.shape)
    #     print(t2.shape)
    # #     # print(len(loop_f))
    # #     # data,a,b,c,d,e,f,g,h,var = loop_f
    # #     # a = chrs[0]
    # #     # b = chrs[1]
    # #     # c = chrs[2]
    # #     # d = chrs[3]
    # #     # e = chrs[4]
    # #     # f = chrs[5]
    # #     # g = chrs[6]
    # #     # h = chrs[7]
    # #     # d = data.clone
    # #     # print(data.device)
    #     break
    # s0 = torch.clone(t1)
    # s1 = torch.clone(t2)
    F32i = torch.LongTensor([F32])[0].to(device)
    print("new")
    # data_n = data.clone()
    # batch_h_n = batch_h.clone()
    # s0,s1,t1,t2 = [var.clone() for x in range(4)]
    a = chrs[0]
    b = chrs[1]
    c = chrs[2]
    d = chrs[3]
    e = chrs[4]
    f = chrs[5]
    g = chrs[6]
    h = chrs[7]
    data = datak#[datak[x] for x in range(len(datak))]
    start = time.process_time()
    for x in range(batch_n//each):
        print(x)
        
        ret = SHA256_hash(data, a,b,c,d,e,f,g,h,k,t1,t2,F32i,consts)
        # data  =torch.clone(data)
        # chrs = torch.clone(chrs)
        # a = chrs[0]
        # b = chrs[1]
        # c = chrs[2]
        # d = chrs[3]
        # e = chrs[4]
        # f = chrs[5]
        # g = chrs[6]
        # h = chrs[7]
        # data = [(datak)[x] for x in range(len(datak))]
        # t1 = torch.clone(t1)
        # t2 = torch.clone(t2)
        # s0 = torch.clone(s0)
        # s1 = torch.clone(s1)
        # a = torch.clone(a)
        # b = torch.clone(b)
        # c = torch.clone(c)
        # d = torch.clone(d)
        # e = torch.clone(e)
        # f = torch.clone(f)
        # g = torch.clone(g)
        # h = torch.clone(h)
        # k = torch.clone(k)
        # F32i = torch.clone(F32i)
        # consts = torch.clone(consts)
        # # print(id(data))
        # # data = my_stack([data])
        # # batch_h = my_stack([batch_h])
        # # data = torch.clone(data)
        # # chrs = torch.clone(chrs)
        # t1 = torch.clone(t1)
        # t2 = torch.clone(t2)
        # print(id(data))
start = time.process_time()
# batch_n = 10000000
# each = 3000000
batches = 2

each = 3000000
batch_n = each*1
# prc = (xmp.spawn(_mp_fn, args=(), nprocs=8,join=True))
i = 0
# for000000 p in prc:

_mp_fn(0)
#     i+=1
#     print(i)
#     p.join()0
time_taken = time.process_time()-start
time_taken/=batch_n
print(batch_n)
print(each)
print("Hash Rate",8/time_taken)

Started 0
Datak
Chrs
t1
t2
done
new
0
3000000
3000000
Hash Rate 94330798.03984928


In [28]:
device = xm.xla_device()

In [29]:
a = torch.LongTensor([1,2,3,4]).to(device)

In [32]:
torch.div(a,2)

tensor([0.2500, 0.5000, 0.7500, 1.0000], device='xla:1')

In [31]:
a

tensor([0, 1, 1, 2], device='xla:1')

In [ ]:
def _rotr(x, y):
    return ((x >> y) | (x << (32 - y))) & F32

def _maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

def _ch(x, y, z):)
        # t2 = torch.clone(t2
    return (x & y) ^ ((~x) & z)


def SHA256_hash(w,chrs):
	for i in range(16, 64):
		s0 = _rotr(w[:,i-15], 7) ^ _rotr(w[:,i-15], 18) ^ _rotr(w[:,i-15], 3)
		s1 = _rotr(w[:,i-2], 17) ^ _rotr(w[:,i-2], 19) ^ _rotr(w[:,i-15], 10)
		w[:,i] = (w[:,i-16] + s0 + w[:,i-7] + s1) & F32
	for i in range(64):
		s0 = _rotr(chrs[:,i%8], 2) ^ _rotr(chrs[:,i%8], 13) ^ _rotr(chrs[:,i%8], 22)
		t2 = s0 + _maj(chrs[:,i%8], chrs[:,(i+1)%8], chrs[:,(i+2)%8])
		s1 = _rotr(chrs[:,(i+4)%8], 6) ^ _rotr(chrs[:,(i+4)%8], 11) ^ _rotr(chrs[:,(i+4)%8], 25)
		t1 = chrs[:,(i+7)%8] + s1 + _ch(chrs[:,(i+4)%8], chrs[:,(i+5)%8], chrs[:,(i+6)%8]) + self._k[i] + w[:,i]
		i+=1
		i = i%8
		chrs[:,(i+4)%8] = (chrs[:,(i+4)%8]+t1) & F32
		chrs[:,i%8] = (t1 + t2) & F32
	return chrs

SyntaxError: ignored

In [ ]:
def _mp_fn(index):
	print("Started",index)
	hash_dataset = hashing_algorithm(index*batch_n, (index+1)*batch_n, base)
	hash_dataloader = DataLoader(hash_dataset, batch_size=each, shuffle=True)

	mp_device_loader = pl.MpDeviceLoader(hash_dataloader, device)

	for data,batch_h in mp_device_loader:
		ret = SHA256_hash(data, batch_h)
		print(ret)
_mp_fn(0)
# xmp.spawn(_mp_fn, args=())